In [1]:
import kmapper as km
from kmapper.plotlyviz import *

In [2]:
import sklearn
#from sklearn import ensemble
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA
# from sklearn.manifold import MDS

# scipy for interpolation
# import scipy 
# from scipy.interpolate import *

In [3]:
# Plotly and Dash
import plotly.graph_objs as go
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

# TODO fix this merging libraries

In [4]:
import networkx
import matplotlib as plt

In [5]:
import psutil
import kaleido

In [10]:
# pwd # this is how to check working directory

'/Users/amba/Documents/GitHub/UMR-TDA-2021/pools_specific_EDA/Lower Pool 4/kmapper'

In [6]:
predicted_df = pd.read_csv("../../../LTRM data/Interpolated data/interpolated_pool_4.csv")

In [7]:
# filter for pool 4 lower
pool4L = predicted_df[predicted_df['LONGITUDE'] <= -92.33]
pool4L = pool4L[pool4L['STRATUM'] != 4]

In [8]:
### Creating three main time spans and two overlapping time spans,
### a total of five time spans

# defining different time periods
time_list = [[*range(1993, 2000+1)], 
             [*range(1998, 2004+1)], 
             [*range(2001, 2013+1)], 
             [*range(2010, 2016+1)], 
             [*range(2014, 2020+1)]]

time_list_names = ['93-00', '98-04', '01-13', '10-16', '14-20']        


all_strata = [1, 2, 3, 4, 5, 6, 7, 9]
season_list = ["SUMMER"] # , "SPRING", "FALL", "WINTER"]

# create list for non empty stratum
pool4L_strata = []

for stratum_int in all_strata:
    if pool4L[pool4L['STRATUM'] == stratum_int].shape[0]!=0:
        pool4L_strata.append(stratum_int)

pool4L_strata = [1, 2]

In [9]:
pool4L_subsets = []
pool4L_subsets_title = []

for time_period_idx in range(len(time_list)):
    for stratum_int in pool4L_strata:
        for season in season_list:
            title = "stratum" + str(stratum_int) + season + time_list_names[time_period_idx]
            pool4L['TITLE'] = title
            pool4L_subsets.append( pool4L[(pool4L['YEAR'].isin(time_list[time_period_idx])) &
                                                       (pool4L['STRATUM'] == stratum_int ) & 
                                                       (pool4L['SEASON'] == season)] )
            pool4L_subsets_title.append(title)
            
print(len(pool4L_subsets))

10


In [10]:
print(pool4L_subsets[0]['TITLE'][0])

stratum1SUMMER93-00


In [24]:
def cluster_fun(df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = [7,7], PERC_OVERLAP = [.5,.5]):
    """

    """
    
#     keys = list(dict_df.keys())
#     print(keys)
#     X = dict_df.get(keys[0])

    title = df["TITLE"].iloc[0]
    
    X = df[["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB","PREDICTED_COND", "PREDICTED_VEL", "PREDICTED_TP", 
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]]
    
    if X.shape[0] < DBSCAN_MIN_SAMPLES:
        #print(X)
        print("Not enough data to cluster in ", title, "_size = ", X.shape[0])
        print("DBSCAN_MIN_SAMPLES", DBSCAN_MIN_SAMPLES)
        return([DBSCAN_MIN_SAMPLES, X.shape[0]])
    
    db = DBSCAN(eps=20, min_samples=2).fit(X)
    # Number of clusters in labels, ignoring noise if present.
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('For the ' + title + 'subset, we have:')
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    
    return(db)

    
def mapper_pca_func(df, DBSCAN_EPSILON = 20, DBSCAN_MIN_SAMPLES = 1, N_CUBES = 7, PERC_OVERLAP = .5,
                   scomplex_bool = False):
    """
    str(df_stratum_season_time_dict_list[1].keys())
    """
    
#     keys = list(dict_df.keys())
#     print(keys)
#     X = dict_df.get(keys[0])

    title = df["TITLE"].iloc[0]
    
    # remove conductivity
    
    X = df[["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB", "PREDICTED_VEL", "PREDICTED_TP", 
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]]
    
    # killian
    
    continuous_variables = ["PREDICTED_WDP", "PREDICTED_SECCHI", "PREDICTED_TEMP", "PREDICTED_DO", 
           "PREDICTED_TURB", "PREDICTED_VEL", "PREDICTED_TP", 
           "PREDICTED_TN", "PREDICTED_SS", "PREDICTED_CHLcal"]
    
    var_to_index = {continuous_variables[i] : i for i in range(len(continuous_variables))}
    projected_vars = continuous_variables
    projected_var_indices = [var_to_index[var] for var in projected_vars]
    
    if X.shape[0]<10:
        #print(X)
        print("Not enough data in ", title, "_size = ", X.shape[0])
        return(X.shape[0])

    # defining clustering and kmapper parameters
    
    # to match up indices in scomplex with the original dataframe X
    X.reset_index(drop = True, inplace = True) 
    
    # create instance of clustering alg
    cluster_alg = sklearn.cluster.DBSCAN(eps=DBSCAN_EPSILON, min_samples=DBSCAN_MIN_SAMPLES, metric='euclidean')

    # Instantiate kepler mapper object
    mapper = km.KeplerMapper(verbose=0)
    
    # defining filter function as projection on to the first 2 component axis
    pca = PCA(n_components=1)
    lens = pca.fit_transform(X)
    
    # killian: PCA1 information
    principle_component = max(abs(pca.components_[0].min()), abs(pca.components_[0].max()))
    max_index = 0
    for i in range(len(pca.components_[0])):
        if abs(pca.components_[0][i]) == principle_component:
            max_index = i
    print("Primary variable: ", continuous_variables[max_index])
    print("Corresponding component: ", pca.components_[0][max_index])
    print('Explained Variance: ', pca.explained_variance_ratio_)
    
    summary_variable = mapper.project(np.array(X), projection=projected_var_indices, scaler=None)
    # similar to fit transform
    
    # Generate the simplicial complex
    scomplex = mapper.map(lens, X, 
                          cover=km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP), 
                          clusterer=cluster_alg)  

    if scomplex_bool: 
        return(scomplex)
    
    pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

    color_values = lens[:,0] - lens[:,0].min() # changes if PCA1 or PCA1 and PCA2
    # can change to other variables
    color_function_name = ['Distance to x-min'] # set name of color function
    my_colorscale = pl_brewer
    kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                     color_values,  
                                                                     color_function_name=color_function_name, 
                                                                     colorscale=my_colorscale)

    bgcolor = 'rgba(10,10,10, 0.9)'
    # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

    plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                     factor_size=2.5, edge_linewidth=0.5)
    plot_title = str(title) + str(DBSCAN_EPSILON) + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    # plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(DBSCAN_EPSILON) + ', MIN_SAMPLES ' + str(DBSCAN_MIN_SAMPLES) 
    layout = plot_layout(title=plot_title,  
                         width=620, height=570,
                         annotation_text=get_kmgraph_meta(mapper_summary),  
                         bgcolor=bgcolor)

    # FigureWidget is responsible for event listeners

    fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
    fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
    fw_summary = summary_fig(mapper_summary, height=300)

    dashboard = hovering_widgets(kmgraph, 
                                 fw_graph, 
                                 bgcolor=bgcolor, 
                                 member_textbox_width=600)

    # DESIRED FILE PATH, CHANGE TO FIT YOUR LOCAL MACHINE
    directory_path = "mapper outputs"
    
    #Update the fw_graph colorbar, setting its title:
    fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
    html_output_path = directory_path + title + 'Eps_' + str(DBSCAN_EPSILON) +'_MinS_' + str(DBSCAN_MIN_SAMPLES) + '_NCubes_' + str(N_CUBES) + '_PercOvlp_' + str(PERC_OVERLAP) + '.html'
    
    mapper.visualize(scomplex, color_values=color_values, color_function_name=color_function_name,
                     path_html=html_output_path, lens = summary_variable, lens_names = projected_vars)
    
    return scomplex, X

In [12]:
dbclus_list = []
for subset_df in pool4L_subsets:
    dbclus_list.append(cluster_fun(subset_df))

For the stratum1SUMMER93-00subset, we have:
Estimated number of clusters: 9
Estimated number of noise points: 5
For the stratum2SUMMER93-00subset, we have:
Estimated number of clusters: 7
Estimated number of noise points: 9
For the stratum1SUMMER98-04subset, we have:
Estimated number of clusters: 5
Estimated number of noise points: 4
For the stratum2SUMMER98-04subset, we have:
Estimated number of clusters: 5
Estimated number of noise points: 7
For the stratum1SUMMER01-13subset, we have:
Estimated number of clusters: 3
Estimated number of noise points: 2
For the stratum2SUMMER01-13subset, we have:
Estimated number of clusters: 4
Estimated number of noise points: 10
For the stratum1SUMMER10-16subset, we have:
Estimated number of clusters: 6
Estimated number of noise points: 1
For the stratum2SUMMER10-16subset, we have:
Estimated number of clusters: 4
Estimated number of noise points: 7
For the stratum1SUMMER14-20subset, we have:
Estimated number of clusters: 5
Estimated number of noise p

In [25]:
# parameter sequences

n_cubes_list = [5, 10, 15]
perc_overlap_list = [.35, .45, .55]

mapper_pca_output = []
mapper_pca_output_title = []
mapper_output_df = []


In [26]:

for subset_df in pool4L_subsets:
    for n_cubes in n_cubes_list:
        for perc_overlap in perc_overlap_list: 
            
            scomplex, X = mapper_pca_func(subset_df, n_cubes, perc_overlap, scomplex_bool = False)
            
            
            mapper_pca_output.append(scomplex)
            mapper_pca_output_title.append(subset_df['TITLE'].iloc[0])
            mapper_output_df.append(X)


Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary variable:  PREDICTED_SS
Corresponding component:  0.736504242934729
Explained Variance:  [0.83489094]
Primary va

In [24]:
scomplex_pca_output = []

for subset_df in pool4L_subsets:
    for n_cubes in n_cubes_list:
        for perc_overlap in perc_overlap_list: 
            scomplex_pca_output.append(mapper_pca_func(subset_df, 
                                                       N_CUBES = n_cubes, 
                                                       PERC_OVERLAP = perc_overlap, 
                                                       scomplex_bool = True))

In [158]:
mapper_pca_output[0]

{'nodes': defaultdict(list,
             {'cube0_cluster0': [1],
              'cube0_cluster1': [2],
              'cube0_cluster2': [3],
              'cube0_cluster3': [4],
              'cube0_cluster4': [7],
              'cube0_cluster5': [8],
              'cube0_cluster6': [19],
              'cube0_cluster7': [20],
              'cube0_cluster8': [21],
              'cube0_cluster9': [22],
              'cube0_cluster10': [23],
              'cube0_cluster11': [24],
              'cube0_cluster12': [25],
              'cube0_cluster13': [26],
              'cube0_cluster14': [27],
              'cube0_cluster15': [28],
              'cube1_cluster0': [0],
              'cube1_cluster1': [1],
              'cube1_cluster2': [2],
              'cube1_cluster3': [3],
              'cube1_cluster4': [4],
              'cube1_cluster5': [5],
              'cube1_cluster6': [6],
              'cube1_cluster7': [7],
              'cube1_cluster8': [8],
              'cube1_cluster9':

In [25]:
scomplex_pca_output[0]

{'nodes': defaultdict(list,
             {'cube0_cluster0': [0, 1, 2, 3, 4, 5, 7],
              'cube0_cluster1': [8],
              'cube0_cluster2': [19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
              'cube1_cluster0': [0, 1, 2, 3, 4, 5, 7],
              'cube1_cluster1': [6],
              'cube1_cluster2': [8],
              'cube1_cluster3': [13,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
               50,
               51,
               52,
               53],
              'cube1_cluster4': [16, 17, 18, 66, 67, 68, 69, 70, 71, 72, 73],
              'cube2_cluster0': [9],
              'cube2_cluster1': [10, 11, 15],
              'cube2_cluster2': [12],
              'cube2_cluster3': [16, 17, 18, 66, 67, 68, 69, 70, 71, 72, 73],
              'cube2_cluster4': [36, 37],
              'cube2_cluster5': [38, 39, 40, 41, 42],
              'cube3_cluster0': [9],
              'cube3

In [26]:
for graph_idx in range(0, 135):
    
    plotlyviz(scomplex_pca_output[graph_idx], title=mapper_pca_output_title[graph_idx], 
              filename = str(graph_idx) + mapper_pca_output_title[graph_idx] + ".png")

In [197]:
plotlyviz(scomplex_pca_output[1], title=mapper_pca_output_title[1], graph_data = True)

    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(180,180,…

In [182]:
subset_df["YEAR"].iloc[0]

2014